In [2]:
# Импортируем Пандас
import pandas as pd
# Загружаем файл в таблицу df (data frame)
df = pd.read_csv('data-1-structure-1.csv')
# Удаляем лишние столбцы. Когда будем удалять строки с пустыми значениями, мы не потеряем те, где есть всё кроме, например, автора, который и так не нужен
df.drop(["Название", "Композитор", "Автор текста", "ID музыкального жанра", "Тема", "Музыкальный жанр"], axis = 1, inplace = True)
# Удаляем строки с пустыми значениями
df.dropna(inplace = True)
# Удаляем строки, где в годах есть тире, нам не нужны временные отрезки
df = df[df["Год создания"].str.contains("-") == False]
# Удаляем строки, где в годах фактически пусто, но технически пробел
df = df[df["Год создания"].str.contains(" ") == False]

# Сортируем по годам
df = df.sort_values(by = ["Год создания"], ascending = True)
# Актуализируем индексы
df = df.reset_index(drop = True)

# Удаляем пробелы в начале и в конце ключевых слов, чтобы "Петр Первый" и "Петр Первый " считать за одно
dfL = df["Ключевые слова"].tolist()
for n, i in enumerate(dfL, 0):
        i = str(i)
        i = i.strip()
        dfL[n] = i
df["Ключевые слова"] = dfL

# Экспортируем df как csv
df.to_csv("S-years-words.csv")
# Экспортируем df как xlsx
df.to_excel("S-years-words.xlsx")

# Печатаем df
df

,Год создания,Ключевые слова
0,1709,Сражение под Полтавой 1709
1,1710,Сражение под Полтавой 1709
2,1710,Сражение под Полтавой 1709
3,1786,богатыри
4,1791,взятие Измаила
...,...,...
3820,2008,Евпатий Коловрат
3821,2009,Урал
3822,2009,Иван Грозный
3823,2009,Чебоксары


In [3]:
# Создаем пустой словарь
a = {}
# Перебираем года из df
for i in df["Год создания"]:
# Если год встречается первый раз, добавляем его в словарь и выставляем счетчик на единицу
  if i not in a:
    a[i] = 1
# Если год уже есть в словаре, увеличиваем его счетчик на единицу
  else:
    a[i] += 1
# Создаем из словаря таблицу years
years = pd.DataFrame(list(a.items()), columns=['Год создания', 'Количество музыкальных произведений'])
# Сортируем по годам
years = years.sort_values(by = ["Год создания"], ascending = True)
# Актуализируем индексы
years = years.reset_index(drop = True)

# Экспортируем years как csv
years.to_csv("S-years.csv")
# Экспортируем years как xlsx
years.to_excel("S-years.xlsx")

# Печатаем years
years

,Год создания,Количество музыкальных произведений
0,1709,1
1,1710,2
2,1786,1
3,1791,1
4,1796,1
...,...,...
137,2006,5
138,2007,2
139,2008,2
140,2009,3


In [4]:
# Импортируем счетчик
from collections import Counter

# Создаем пустой словарь
v = {}
# Перебираем все года
for i in df["Год создания"]:
  # Создаем таблицу для каждого отдельного года
  dfi = df[df["Год создания"].str.contains(i)]
  # Каждому году в словаре создаем значение "NaN"
  v[i] = "NaN"
  # Обращаемся к каждому ключевому слову в таблице для каждого отдельного года
  for k in dfi["Ключевые слова"]:
    # Если год встречается впервые, создаем пустой список bb и добавляем к нему первое слово в этом году
    if v[i] == "NaN":
      bb = []
      bb.append(k)
      # Меняем значение года с "NaN" на список слов bb, который создали выше
      v[i] = bb
    # Если у года уже есть связанный с ним список слов bb (хоть даже из одного слова), добавляем новое слово в этот список
    else:
      bb.append(k)
      v[i] = bb

# Перебираем каждый элемент сформированного словаря
for i in v:
  # Называем списки слов для каждого года переменной x
  x = v[i]
  # Если для года нет такого списка, оставляем все как было
  if x == "NaN":
    x = "NaN"
  # В остальных случаях находим слово в списке, которое встречается чаще всего, и связываем его с годом, заменяя им список слов 
  else:
    xm = [word for word, word_count in Counter(x).most_common(1)]
    v[i] = xm

# Создаем из словаря таблицу words
words = pd.DataFrame(list(v.items()), columns=['Год создания', 'Самый популярный тэг'])
# Сортируем по годам
words = words.sort_values(by = ["Год создания"], ascending = True)
# Актуализируем индексы
words = words.reset_index(drop = True)

# Экспортируем words как csv
words.to_csv("S-words.csv")
# Экспортируем words как xlsx
words.to_excel("S-words.xlsx")

# Печатаем words
words

,Год создания,Самый популярный тэг
0,1709,[Сражение под Полтавой 1709]
1,1710,[Сражение под Полтавой 1709]
2,1786,[богатыри]
3,1791,[взятие Измаила]
4,1796,[строевая]
...,...,...
137,2006,[Россия]
138,2007,[Чувашия]
139,2008,[Саранск]
140,2009,[Урал]


In [5]:
# Объединяем таблицы years и words в таблицу wy
wy = years.merge(words)

# Делаем тэги чистыми на вид, без ['']
wyL = wy["Самый популярный тэг"].tolist()
for n, i in enumerate(wyL, 0):
        i = str(i)
        i = i[2:-2]
        wyL[n] = i
wy["Самый популярный тэг"] = wyL

# Экспортируем wy как csv
wy.to_csv("SaC-wy.csv")
# Экспортируем wy как xlsx
wy.to_excel("SaC-wy.xlsx")

# Печатаем wy
wy

,Год создания,Количество музыкальных произведений,Самый популярный тэг
0,1709,1,Сражение под Полтавой 1709
1,1710,2,Сражение под Полтавой 1709
2,1786,1,богатыри
3,1791,1,взятие Измаила
4,1796,1,строевая
...,...,...,...
137,2006,5,Россия
138,2007,2,Чувашия
139,2008,2,Саранск
140,2009,3,Урал
